In [1]:
import hazm
import numpy as np
import pandas as pd
from hazm import word_tokenize,Normalizer
import nltk
from nltk.tokenize import word_tokenize as eng_tokenize
from tqdm.auto import tqdm
import pickle
import json
import string
import re
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
# next(iter(imdb_dataset))["translation"]
dataset=load_dataset("tep_en_fa_para")["train"]["translation"]

In [3]:
def clean(farsi_s,eng_s):
    farsi_s=farsi_s.lower()
    farsi_s=re.sub(r",+"," , ",farsi_s)
    farsi_s=re.sub(r"\.+"," . ",farsi_s)
    farsi_s=re.sub(r"\_+"," _ ",farsi_s)
    farsi_s=re.sub(r"\$","",farsi_s)
    farsi_s=re.sub(r"#","",farsi_s)
    farsi_s=re.sub(r"=","",farsi_s)
    farsi_s=re.sub(r"@","",farsi_s)
    farsi_s=re.sub(r"\d","",farsi_s)
    farsi_s=re.sub(r"~","",farsi_s)
    farsi_s=re.sub(r"\'","",farsi_s)
    farsi_s=re.sub(r">","",farsi_s)
    farsi_s=re.sub(r"<","",farsi_s)
    farsi_s=re.sub(r"\+","",farsi_s)
    farsi_s=re.sub(r"\-","",farsi_s)
    farsi_s=re.sub(r"\/","",farsi_s)
    farsi_s=re.sub(r"\*","",farsi_s)
    farsi_s=re.sub(r"\"","",farsi_s)
    farsi_s=re.sub(r"\…+"," ",farsi_s)

    eng_s=eng_s.lower()
    eng_s=re.sub(r",+"," , ",eng_s)
    eng_s=re.sub(r"\.+"," . ",eng_s)
    eng_s=re.sub(r"\_+"," _ ",eng_s)
    eng_s=re.sub(r"\$","",eng_s)
    eng_s=re.sub(r"#","",eng_s)
    eng_s=re.sub(r"=","",eng_s)
    eng_s=re.sub(r"@","",eng_s)
    eng_s=re.sub(r"\d","",eng_s)
    eng_s=re.sub(r"~","",eng_s)
    eng_s=re.sub(r"\'","",eng_s)
    eng_s=re.sub(r">","",eng_s)
    eng_s=re.sub(r"<","",eng_s)
    eng_s=re.sub(r"\+","",eng_s)
    eng_s=re.sub(r"\-","",eng_s)
    eng_s=re.sub(r"\/","",eng_s)
    eng_s=re.sub(r"\*","",eng_s)
    eng_s=re.sub(r"\"","",eng_s)
    return farsi_s,eng_s


In [4]:
dataset_en=[]
dataset_fa=[]
vocab_en=[]
vocab_fa=[]
Max_input_size=10
normalizer = Normalizer()
for dic in tqdm(dataset):
    farsi_s=dic["fa"]
    eng_s=dic["en"]
    farsi_s,eng_s=clean(farsi_s,eng_s)
    eng_tokenized=eng_tokenize(eng_s)
    fa_tokenized=word_tokenize(normalizer.normalize(farsi_s))
    if(len(eng_tokenized)>=10 or len(fa_tokenized)>=10):
        continue
    dataset_en.append(eng_tokenized+["_EOS_"])
    dataset_fa.append(fa_tokenized+["_EOS_"])
    vocab_en+=eng_tokenized
    vocab_fa+=fa_tokenized

  0%|          | 0/612087 [00:00<?, ?it/s]

In [5]:
vocab_en=["_EOS_","_PAD_"]+list(set(vocab_en))
vocab_fa=["_EOS_","_PAD_"]+list(set(vocab_fa))

In [9]:
en_index={}
fa_index={}
for i,token in tqdm(enumerate(vocab_en)):
    en_index[token]=i
for i,token in tqdm(enumerate(vocab_fa)):
    fa_index[token]=i

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [15]:
with open("./hugg_preprocessed/en_index.json", "w") as outfile: 
    json.dump(en_index, outfile)
with open("./hugg_preprocessed/fa_index.json", "w") as outfile: 
    json.dump(fa_index, outfile)
with open("./hugg_preprocessed/vocab_en", "wb") as fp:   
    pickle.dump(vocab_en, fp)
with open("./hugg_preprocessed/vocab_fa", "wb") as fp:  
    pickle.dump(vocab_fa, fp)
with open("./hugg_preprocessed/dataset_en", "wb") as fp:   
    pickle.dump(dataset_en, fp)
with open("./hugg_preprocessed/dataset_fa", "wb") as fp:
    pickle.dump(dataset_fa, fp)

In [13]:
with open('en_index.json',"r") as json_file:
    data = json.load(json_file)